In [1]:
import random
from IPython.display import display, Latex, Markdown



# Baby Kyber https://cryptopedia.dev/posts/kyber/


## Parameter

Alle Parameter können frei angepasst werden.
Die Standardwerte sollten zu übersichtlichkeit klein gehalten werden.


In [2]:

# ===== Parameter =====
q = 17              # Modulus für Koeffizienten
n = 4               # Grad: Polynomring Z_q[x] / (x^n + 1)
k = 2               # Vektorlänge
kleine_faktoren = [-1, 0, 1] 

q_halbe = (q + 1) // 2

print("q =", q)
print("n =", n)
print("k =", k)
print("⌊q/2⌉ =", q_halbe)


q = 17
n = 4
k = 2
⌊q/2⌉ = 9



## Polynom-Arithmetik

Polynome werden als Listen dargestellt:
[a0, a1, a2, a3]  ↔  a0 + a1·x + a2·x² + a3·x³


In [3]:
def poly_add(a, b):
    return [(x + y) % q for x, y in zip(a, b)]

def poly_sub(a, b):
    return [(x - y) % q for x, y in zip(a, b)]

def poly_mul(a, b):
    res = [0] * (2*n)
    for i in range(n):
        for j in range(n):
            res[i+j] += a[i] * b[j]
    # Reduktion modulo x^n + 1
    for i in range(n, 2*n):
        res[i-n] -= res[i]
    return [x % q for x in res[:n]]

def zufalls_polynom(klein=True):
    if klein:
        return [random.choice(kleine_faktoren) for _ in range(n)]
    return [random.randrange(q) for _ in range(n)]

def poly_str(p):
    """Formatiert ein Polynom als lesbare String mit x^i Notation"""
    terms = []
    for i, coeff in enumerate(p):
        if coeff == 0:
            continue
        
        if i == 0:
            terms.append(str(coeff))
        elif i == 1:
            if coeff == 1:
                terms.append("x")
            else:
                terms.append(f"{coeff}·x")
        else:
            if coeff == 1:
                terms.append(f"x^{i}")
            else:
                terms.append(f"{coeff}·x^{i}")
    
    if not terms:
        return "0"
    return " + ".join(terms)

def matrix_latex(M):
    """Formatiert eine Matrix als LaTeX und gibt Latex-Objekt zurück"""
    rows = []
    for row in M:
        row_str = " & ".join([poly_str(p) for p in row])
        rows.append(row_str)
    matrix_str = " \\\\ ".join(rows)
    latex_str = f"\\begin{{pmatrix}} {matrix_str} \\end{{pmatrix}}"
    return Latex(latex_str)

def vector_latex(v):
    """Formatiert einen Vektor als LaTeX und gibt Latex-Objekt zurück"""
    row_str = " \\\\ ".join([poly_str(p) for p in v])
    latex_str = f"\\begin{{pmatrix}} {row_str} \\end{{pmatrix}}"
    return Latex(latex_str)

def poly_latex(v):
    """Formatiert ein Polynom als zentriertes LaTeX-Objekt"""
    latex_str = r"\[ " + str(v) + r" \]"
    return Latex(latex_str)


## Schlüsselgenerierung durch Alice


In [4]:
# Privater Schlüssel s
s = [zufalls_polynom(klein=True) for _ in range(k)]

# Fehlervektor e
e = [zufalls_polynom(klein=True) for _ in range(k)]

# Öffentliche Matrix A
A = [[zufalls_polynom(klein=False) for _ in range(k)] for _ in range(k)]

display(Markdown("**Zufällig generierter Privater Schlüssel s:**"))
display(vector_latex(s))

display(Markdown("**Zufällig generierter Fehlervektor e:**"))
display(vector_latex(e))

display(Markdown("**Zufällig generierte Öffentliche Matrix A:**"))
display(matrix_latex(A))

# t = A*s + e
def mat_vec_mul(A, v):
    out = []
    for row in A:
        acc = [0]*n
        for a, b in zip(row, v):
            acc = poly_add(acc, poly_mul(a, b))
        out.append(acc)
    return out

t = [poly_add(x, y) for x, y in zip(mat_vec_mul(A, s), e)]

display(Markdown(r"**Öffentlicher Vektor t**"))
display(Markdown(r"\\[ \mathbf{t} = A\mathbf{s} + \mathbf{e} \\]"))
display(vector_latex(t))


**Zufällig generierter Privater Schlüssel s:**

<IPython.core.display.Latex object>

**Zufällig generierter Fehlervektor e:**

<IPython.core.display.Latex object>

**Zufällig generierte Öffentliche Matrix A:**

<IPython.core.display.Latex object>

**Öffentlicher Vektor t**

\\[ \mathbf{t} = A\mathbf{s} + \mathbf{e} \\]

<IPython.core.display.Latex object>


## Verschlüsselung durch Bob


In [22]:
# Zufälliger Vektor r und Fehler
r = [zufalls_polynom(klein=True) for _ in range(k)]
e1 = [zufalls_polynom(klein=True) for _ in range(k)]
e2 = zufalls_polynom(klein=True)

display(Markdown("**Zufallsvektor r:**"))
display(vector_latex(r))

display(Markdown("**Zufällig generierter Fehler e1:**"))
display(vector_latex(e1))

display(Markdown("**Zufällig generierter Fehler e2**:"))
display(poly_latex(poly_str(e2)))

# Nachricht (zufällige 4 Bit)
m_bin = [random.randint(0,1) for _ in range(n)]
m = [(q_halbe * x) % q for x in m_bin]

display(Markdown("**Binäre Nachricht m_bin:**"))
display(poly_latex(m_bin))
display(Markdown("**Skalierte Nachricht m:**"))
display(poly_latex(poly_str(m)))

# u = A^T r + e1
AT = list(zip(*A))
u = [poly_add(acc, err) for acc, err in zip(mat_vec_mul(AT, r), e1)]

# v = t^T r + e2 + m
v = [0]*n
for ti, ri in zip(t, r):
    v = poly_add(v, poly_mul(ti, ri))
v = poly_add(poly_add(v, e2), m)

display(Markdown("**Ciphertext u:**"))
display(poly_latex("u = A^T r + e1"))
display(vector_latex(u))

display(Markdown("**Ciphertext v**:"))
display(poly_latex("v = t^T r + e2 + m"))
display(poly_latex(poly_str(v)))


**Zufallsvektor r:**

<IPython.core.display.Latex object>

**Zufällig generierter Fehler e1:**

<IPython.core.display.Latex object>

**Zufällig generierter Fehler e2**:

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

**Ciphertext v**:

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>


## Entschlüsselung durch Alice


In [29]:
# m_noisy = v - s^T u
m_noisy = v.copy()
for si, ui in zip(s, u):
    m_noisy = poly_sub(m_noisy, poly_mul(si, ui))

display(Markdown("**Rauschbehaftete Nachricht m~**"))
display(poly_latex(poly_str(m_noisy)))

# Rundung
m_rec = []
for c in m_noisy:
    if abs(c - q_halbe) < abs(c):
        m_rec.append(1)
    else:
        m_rec.append(0)

display(Markdown("**Wiederhergestellte Bits**"))
display(poly_latex(m_rec))
display(Markdown("**Originale Bits**"))
display(poly_latex(m_bin))


**Rauschbehaftete Nachricht m~**

<IPython.core.display.Latex object>

**Wiederhergestellte Bits**

<IPython.core.display.Latex object>

**Originale Bits**

<IPython.core.display.Latex object>


## Zusammenfassung

- Alle Werte zufällig erzeugt
- Jeder Zwischenschritt ausgegeben
- Klassische Baby-Kyber-Darstellung
- Ideal zum Nachrechnen auf Papier

So wurde Kryptographie traditionell erklärt: klein, offen, nachvollziehbar.
